# Accessing TTree data with PyROOT

Very likely, most of your data are in ROOT TTrees. Here are various ways of getting the data out.

In [1]:
import ROOT

Welcome to JupyROOT 6.14/04


In [2]:
# Note: this is TFile::Open(...)
file = ROOT.TFile.Open("http://scikit-hep.org/uproot/examples/HZZ.root")
tree = file.Get("events")

In [3]:
for event in tree:
    print(event.MET_px, event.MET_py)

5.912771224975586 2.5636332035064697
24.76520347595215 -16.349109649658203
-25.78508758544922 16.237131118774414
8.619895935058594 -22.78654670715332
5.393138885498047 -1.3100523948669434
-3.7594752311706543 -19.417020797729492
23.962148666381836 -9.049156188964844
-57.533348083496094 -20.48767852783203
42.416194915771484 -94.35086059570312
-1.9144694805145264 -23.96303367614746
19.710058212280273 4.645508766174316
-35.538055419921875 -14.753822326660156
15.2099027633667 -6.3974995613098145
28.45915985107422 5.657520294189453
-9.093977928161621 0.8614864945411682
0.3503572940826416 -4.255575656890869
-0.20190052688121796 -3.49951434135437
8.286861419677734 -3.2661635875701904
-7.835591793060303 -13.773303031921387
-41.88085174560547 -157.98814392089844
-1.7157256603240967 19.48651885986328
2.3737385272979736 -8.089887619018555
-30.23499298095703 2.0271899700164795
13.711685180664062 -6.066513538360596
130.72396850585938 74.9982681274414
-6.254471778869629 18.144054412841797
-14.4688358

-17.536632537841797 18.48490333557129
-8.252344131469727 -4.563766956329346
14.926718711853027 18.703094482421875
-183.2861785888672 -41.52691650390625
-80.09735870361328 1.8012679815292358
22.565441131591797 -39.01847839355469
23.564111709594727 -24.29120445251465
48.90098571777344 15.634029388427734
-1.7056713104248047 17.405439376831055
13.855692863464355 -2.0340075492858887
14.49750804901123 7.248403072357178
-9.844893455505371 53.30706024169922
-8.066815376281738 13.305007934570312
8.482353210449219 -2.834740400314331
35.38676452636719 14.327507972717285
14.389554977416992 11.93272876739502
43.51626968383789 71.00537872314453
-15.688297271728516 3.6851320266723633
16.5919132232666 15.45226001739502
7.9199628829956055 -2.8519561290740967
58.81696319580078 -57.995460510253906
14.860068321228027 -8.082831382751465
14.2335786819458 -21.44774055480957
20.75125503540039 -18.66448402404785
-19.637371063232422 5.06316614151001
-12.574421882629395 54.02536392211914
-14.587543487548828 -16.

That worked, but it would be uncomfortable on large datasets because expressions like `event.MET_px` does a lot of work _in each event_ to produce a number. The same reasons to use Numpy over Python for loops applies here to an even greater degree.

Fortunately, ROOT now has methods to access TTree data as Numpy arrays.

In [5]:
tree.AsMatrix(["MET_px", "MET_py"])   # , return_labels=True

array([[  5.91277122,   2.5636332 ],
       [ 24.76520348, -16.34910965],
       [-25.78508759,  16.23713112],
       ...,
       [ 18.10164642,  50.29071808],
       [ 79.87519073, -52.35145187],
       [ 19.71374893,  -3.59541821]])